In [9]:
%load_ext autoreload
%autoreload 2

from source.read_HAR_dataset import read_har_dataset, har_dimensions, har_activities, har_activities_map, har_ind_IDS
from source.utils import  filter_dimensions
from source.tserie import TSerie
from source.utils import classify_dataset
from itertools import chain, combinations
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn import svm
from source.utils import idsStd
from sklearn import metrics
import numpy as np
from sklearn import svm
import umap
from imblearn.under_sampling import RandomUnderSampler
from source.augmentation import  * 
# from cuml.datasets import make_blobs
# from cuml.neighbors import NearestNeighbors
# from cuml.manifold import UMAP
# from cuml.cluster import DBSCAN


sys.path.insert(0, '/home/texs/Documentos/Repositories/mts_viz')
from server.source.storage import MTSStorage

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

N_COMPONENTS=64
Z_SCORE_NORM = True
DATASET = 'HAR-UML20'
KFOLDS = 1
N_TESTS = 2
METRIC  = 'braycurtis'
RESULTS_PATH = 'outputs/augmentation/'
AUGMENTATIONS = ['none', 'rotation', 'permutation', 'time_warp', 'magnitude_warp', 'scaling', 'jitter']
# AUGMENTATIONS = ['none']
# AUGMENTATIONS = ['scaling']
ALL_AUGMENTATIONS = ['none', 'rotation', 'permutation', 'time_warp', 'magnitude_warp', 'scaling', 'jitter']
# AUGMENTATIONS = ['magnitude_warp']
REPEATS_PER_AUGMENTATION = 1
INCLUDE_ORIGINAL = True
# N_DIMS_NAMES = ['Acc', 'Gyro', 'Mag']
N_DIMS_NAMES = ['Acc', 'Gyro']
# N_DIMS_NAMES = ['Acc']
N_DIMENSIONS = [
    [
        'Accelerometer-X',	
        'Accelerometer-Y',	
        'Accelerometer-Z',
    ],
    [
        'Gyrometer-X',
        'Gyrometer-Y',
        'Gyrometer-Z',
    ],
    # [
    #     'Magnetometer-X',
    #     'Magnetometer-Y',
    #     'Magnetometer-Z'
    # ]
]

firstTimeSave = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def center(X_train, X_test, train_ind, test_ind, dimensions ):
    N_tr, T, D = X_train.shape
    N_te, T, D = X_test.shape
    X_train_sh = np.zeros(X_train.shape)
    X_test_sh = np.zeros(X_test.shape)
    for i in range(N_tr):
        
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_train[i, :, k], axis = 0)
                indice = np.where(train_ind ==  I_train[i])[0][0]
                std = ind_std_train[indice][k] * 6
                X_train_sh[i, :, k] = (X_train[i, :, k] - mag)
                # X_train_sh[i, :, k] = np.concatenate([[0], fft(X_train[i, :, k])[1:]])
            else:
                X_train_sh[i, :, k] = X_train[i, :, k]

    for i in range(N_te):
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_test[i, :, k], axis = 0)
                indice = np.where(test_ind ==  I_test[i])[0][0]
                std = ind_std_test[indice][k] * 6
                X_test_sh[i, :, k] = (X_test[i, :, k] - mag)
                # X_test_sh[i, :, k] = fft(X_test[i, :, k])[0:]
            else:
                X_test_sh[i, :, k] = X_test[i, :, k]
    return X_train_sh, X_test_sh

def znorm(X_train, X_test, train_ind, test_ind, ind_std_train, ind_std_test, dimensions):
    N_tr, T, D = X_train.shape
    N_te, T, D = X_test.shape
    X_train_sh = np.zeros(X_train.shape)
    X_test_sh = np.zeros(X_test.shape)
    for i in range(N_tr):
        
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_train[i, :, k], axis = 0)
                indice = np.where(train_ind ==  I_train[i])[0][0]
                std = ind_std_train[indice][k] * 6
                X_train_sh[i, :, k] = (X_train[i, :, k] - mag) / std
            else:
                X_train_sh[i, :, k] = X_train[i, :, k]

    for i in range(N_te):
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_test[i, :, k], axis = 0)
                indice = np.where(test_ind ==  I_test[i])[0][0]
                std = ind_std_test[indice][k] * 6
                X_test_sh[i, :, k] = (X_test[i, :, k] - mag) / std
            else:
                X_test_sh[i, :, k] = X_test[i, :, k]
    return X_train_sh, X_test_sh

def augmentData(X, y, augmentation, repeat = 3):
    X_out = []
    y_out = []
    for i in range(repeat):
        if augmentation == 'rotation':
            augmented = rotation(X, angle_range=[-np.pi/4, np.pi/4])
        elif augmentation == 'permutation':
            augmented = permutation(X)
        elif augmentation == 'time_warp':
            augmented = time_warp(X, sigma=0.1)
        elif augmentation == 'magnitude_warp':
            augmented = magnitude_warp(X, sigma=0.2, knot=4)
        elif augmentation == 'scaling':
            augmented = scaling(X, sigma=0.1)
        elif augmentation == 'jitter':
            augmented = jitter(X, sigma=0.02)
        else:
            augmented = X.copy()
        if len(X_out) == 0:
            X_out = augmented
            y_out = y.copy()
        else:
            X_out = np.concatenate((X_out, augmented), axis=0)
            y_out = np.concatenate((y_out, y), axis=0)
    return X_out, y_out

def augment(X, y, augmentations, repeats_per_augmentation=1, include_original=False):
    X_aug = []
    y_aug = []
    if include_original:
        X_aug = X.copy()
        y_aug = y.copy()
    for augmentation in augmentations:
        curr_X_aug, curr_y_aug = augmentData(X, y, augmentation, repeat=repeats_per_augmentation)
        if len(X_aug) == 0:
            X_aug = curr_X_aug
            y_aug = curr_y_aug
        else:
            X_aug = np.concatenate((X_aug, curr_X_aug), axis=0)
            y_aug = np.concatenate((y_aug, curr_y_aug), axis=0)
    return X_aug, y_aug
        
def minoritySampling(X, y):
    rus = RandomUnderSampler(sampling_strategy='not minority', random_state=1)
    N, T, D = X.shape
    X_temp = X.reshape([N, T * D])
    X_temp, y = rus.fit_resample(X_temp, y)
    X = X_temp.reshape([X_temp.shape[0], T, D])
    return X, y

In [11]:
from source.torch_utils import getContrastiveFeatures
import torch

storage = MTSStorage('har_augmentations')
storage.delete()
storage.load()


components_map = {}

for k in range(KFOLDS):
    # ------------------------ Reading the dataset ------------------------
    if DATASET == 'HAR-UML20':
        all_ids = har_ind_IDS
        test_ids = all_ids[k: k + N_TESTS]
        train_ids = all_ids[:k] + all_ids[k + N_TESTS:]        
        
        data = read_har_dataset('./datasets/HAR-UML20/', train_ids=train_ids, test_ids=test_ids, val_ids=[], cache=True)
        ids_train, X_train, y_train, I_train, train_kcal_MET = data['train']
        # ids_val, X_val, y_val, I_val, val_kcal_MET = data['val']
        ids_test, X_test, y_test, I_test, test_kcal_MET = data['test']
        
        har_activities_map = {
            0: "Sedentary",
            1: "Walking",
            2: "Running",
            3: "Downstairs",
            4: "Upstairs"
        }

        
        all_dimensions = har_dimensions
        activities_map = har_activities_map
        
        y_train[y_train==0] = 0
        y_train[y_train==1] = 0
        y_train[y_train==2] = 0
        y_test[y_test==0] = 0
        y_test[y_test==1] = 0
        y_test[y_test==2] = 0

        for i in range(3, len(har_activities)):
            y_train[y_train==i] = i - 2
            y_test[y_test==i] = i - 2
        
        ind_std_train = idsStd(train_ids , X_train, I_train)
        ind_std_test = idsStd(test_ids, X_test, I_test)
        
        # unique, counts = np.unique(y_train, return_counts=True)
        # unique, counts = np.unique(y_test, return_counts=True)
        X_train, y_train = minoritySampling(X_train, y_train)
        X_test, y_test = minoritySampling(X_test, y_test)
        X_test_o = X_test.copy()
        y_test_o = y_test.copy()
        
    # ---------------------------------------------------------------------

    
    # --------------------------------AugmentData ---------------------------------------------------
    if Z_SCORE_NORM:
        # X_train, X_test = center(X_train, X_test, train_ids, test_ids, dimensions = [0, 1, 2])
        X_train, X_test = znorm(X_train, X_test, train_ids, test_ids, ind_std_train, ind_std_test, dimensions = [0, 1, 2, 3, 4, 5])
        X_test_o = X_test.copy()
    
    X_train, y_train = augment(X_train, y_train, repeats_per_augmentation = REPEATS_PER_AUGMENTATION, augmentations = AUGMENTATIONS, include_original = INCLUDE_ORIGINAL)
    
    additional = 1 if INCLUDE_ORIGINAL else 0
    # X_train = np.repeat(X_train, REPEATS_PER_AUGMENTATION * len(AUGMENTATIONS) + additional, axis=0)
    # y_train = np.repeat(y_train, REPEATS_PER_AUGMENTATION * len(AUGMENTATIONS) + additional, axis=0)
    I_train = np.repeat(I_train, REPEATS_PER_AUGMENTATION * len(AUGMENTATIONS) + additional, axis=0)
    # X_train = np.concatenate([X_train, X_train], axis=0)
    # y_train = np.concatenate([y_train, y_train], axis=0)
    # I_train = np.concatenate([I_train, I_train], axis=0)
    
    
    # -----------------------------------------------------------------------------------------------

    
    train_embeddings = []
    test_embeddings = []
    for t in range(len(N_DIMENSIONS)):    
        dimensions = N_DIMENSIONS[t]
        X_train_f = filter_dimensions(X_train, all_dimensions, dimensions)
        X_test_f = filter_dimensions(X_test, all_dimensions, dimensions)
        X_test_f_o = filter_dimensions(X_test_o, all_dimensions, dimensions)
        
        mts_train = TSerie(X = X_train_f, y = y_train, I = I_train, dimensions = dimensions, classLabels=activities_map)
        mts_test = TSerie(X = X_test_f, y = y_test, I = I_test, dimensions = dimensions, classLabels=activities_map)
        
        mts_train.folding_features_v2()
        mts_test.folding_features_v2()
        
        n_neighbors = 15
        
        # reducer = umap.UMAP(n_components=N_COMPONENTS, metric=METRIC, n_neighbors=n_neighbors)
        # embeddings_train = reducer.fit_transform(mts_train.features, y=mts_train.y)
        # embeddings_test = reducer.transform(mts_test.features)
        
        test_tranformations = [
            augment(X_test_f_o.copy(), y_test, repeats_per_augmentation = 1, augmentations = [ALL_AUGMENTATIONS[i]], include_original = False)
            for i in range(len(ALL_AUGMENTATIONS))
        ]
        
        for i in range(len(test_tranformations)):
            # X_trans, y_trans =  augment(X_test_f_o.copy(), y_test, repeats_per_augmentation = 1, augmentations = [ALL_AUGMENTATIONS[i]], include_original = False)
            test_tranformations[i] = test_tranformations[i][0].copy()
            test_tranformations[i] = test_tranformations[i].transpose([0, 2, 1])
            if firstTimeSave:
                # print(np.random.random([test_tranformations[i].shape[0], 2]).shape)
                storage.add_mts(
                    'test {}_{}'.format(ALL_AUGMENTATIONS[i], N_DIMS_NAMES[t]),
                    test_tranformations[i].transpose([0, 2, 1]), 
                    dimensions,
                    coords={
                        'umap': np.random.random([test_tranformations[i].shape[0], 2])
                    }, 
                    labels={
                        'activities': mts_test.y, 
                        # 'participants': mts_test.I
                    }, 
                    labelsNames={'activities': activities_map },
                    sampling = True,
                    n_samples = 400
                )
                storage.save()
        
                
        
        embeddings_train, test_embes  = getContrastiveFeatures(mts_train.X.transpose([0, 2, 1]), mts_train.y, epochs =4, loss_metric='SupConLoss', X_test=test_tranformations)
        
        test_map={}
        for i in range(len(ALL_AUGMENTATIONS)):
             test_map[ALL_AUGMENTATIONS[i]] = test_embes[i]
        
        # ['none', 'rotation', 'permutation', 'time_warp', 'magnitude_warp', 'scaling', 'jitter']
        train_embeddings.append(embeddings_train)
        test_embeddings.append(test_map)
        
        torch.cuda.empty_cache()
        
        mts_train.features = embeddings_train
        # mts_test.features = embeddings_test
        
        # reducer = UMAP(n_components=N_COMPONENTS, n_neighbors=n_neighbors)
        # nn = NearestNeighbors(n_neighbors=n_neighbors, metric=METRIC)
        # nn.fit(mts_train.features)
        # knn_graph = nn.kneighbors_graph(mts_train.features, mode="distance")
        # embeddings_train = reducer.fit_transform(mts_train.features, y=mts_train.y, knn_graph=knn_graph.tocsr(), convert_dtype=True)
        
        # knn_graph2 = nn.kneighbors_graph(mts_test.features, mode="distance")
        # embeddings_test = reducer.transform(mts_test.features, knn_graph=knn_graph2.tocsr())
        
        # train_embeddings.append(embeddings_train)
        # test_embeddings.append(embeddings_test)
        # reducer = umap.UMAP(n_components=2, metric=METRIC, n_neighbors=n_neighbors)
        # coords_train = reducer.fit_transform(mts_train.features, y=mts_train.y)
        # coords_test = reducer.transform(mts_test.features)
        
        

        # reducer = UMAP(n_components=2, n_neighbors=n_neighbors)
        # nn = NearestNeighbors(n_neighbors=n_neighbors, metric=METRIC)
        # nn.fit(mts_train.features)
        # knn_graph = nn.kneighbors_graph(mts_train.features, mode="distance")
        # coords_train = reducer.fit_transform(mts_train.features, y=mts_train.y, knn_graph=knn_graph.tocsr(), convert_dtype=False)
        
        # knn_graph2 = nn.kneighbors_graph(mts_test.features, mode="distance")
        # coords_test = reducer.transform(mts_test.features,  knn_graph=knn_graph2.tocsr())
        

        # indMapTrain = {int(ind): 'sub_' + str(ind) for ind in np.unique(mts_train.I)}
        # indMapTest = {int(ind): 'sub_' + str(ind) for ind in np.unique(mts_test.I)}
        
        # storage.add_mts(
        #     'train_{}_{}'.format(' '.join(AUGMENTATIONS), ' '.join(dimensions)), 
        #     mts_train.X,
        #     dimensions,
        #     coords={'umap': coords_train}, 
        #     labels={
        #         'activities': mts_train.y, 
        #         # 'participants': mts_train.I
        #     }, 
        #     labelsNames={'activities': activities_map, 'participants': indMapTrain},
        #     sampling = True,
        #     n_samples = 400
        # )
                
        # storage.add_mts(
        #     'test_{}_{}'.format(' '.join(AUGMENTATIONS), ' '.join(dimensions)), 
        #     mts_test.X, 
        #     dimensions,
        #     coords={'umap': coords_test}, 
        #     labels={
        #         'activities': mts_test.y, 
        #         # 'participants': mts_test.I
        #     }, 
        #     labelsNames={'activities': activities_map, 'participants': indMapTest},
        #     sampling = True,
        #     n_samples = 400
        # )
        
        storage.save()
    firstTimeSave = False
    
    names_comb = []
    embeddings_comb = []
    for i, combo in enumerate(powerset(list(range(len(N_DIMS_NAMES)))), 1):
        indexes = list(combo)
        name = ''
        train_embedding = []
        test_embedding = {}
        if len(indexes) == 0:
            continue
        for ind in indexes:
            name = name + ' ' + N_DIMS_NAMES[ind]
            if len(train_embedding) == 0:
                train_embedding = train_embeddings[ind]
                for aug in ALL_AUGMENTATIONS:
                    test_embedding[aug] = test_embeddings[ind][aug]
            else:
                train_embedding = np.concatenate([train_embedding, train_embeddings[ind]], axis=1)
                for aug in ALL_AUGMENTATIONS:
                    test_embedding[aug] = np.concatenate([test_embedding[aug], test_embeddings[ind][aug]], axis=1)        
        names_comb.append(name)
        embeddings_comb.append((train_embedding, test_embedding))
    
    for j in range(len(names_comb)):
        print(j)
        name = names_comb[j]
        # clf = AdaBoostClassifier()
        # clf = XGBClassifier()
        # clf = svm.SVC()
        clf = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')
        train_feat, test_feat_map = embeddings_comb[j]
        clf.fit(train_feat, mts_train.y)
        
        pred_train = clf.predict(train_feat)
        f1_tr = metrics.f1_score(mts_train.y, pred_train, average='weighted')
        
        f1_scores = [f1_tr]
        
        for aug in ALL_AUGMENTATIONS:
            test_feat = test_feat_map[aug]
            pred_test = clf.predict(test_feat)
            f1_te = metrics.f1_score(y_test_o, pred_test, average='weighted')
            f1_scores.append(f1_te)
        
        if name not in components_map:
            components_map[name] = [f1_scores]
        else:
            components_map[name] = components_map[name] + [f1_scores]
    



    

Train IDS: [2, 3, 4, 5, 6, 7, 8, 9]
Test IDS: [0, 1]
Val IDS: []
Loading dataset from cache...


/home/texs/Documentos/Repositories/mts_feature_learning/source/augmentation.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  warp = np.concatenate(np.random.permutation(splits)).ravel()
<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
Subsequence length: 180
Epoch[1] Train loss    avg: 3.6633144005669487
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
mts shape: N: 1500 -  T: 200 - D: 3
Subsequence length: 180
Epoch[1] Train loss    avg: 3.758205520417955
0
1
2


XGBoostError: [10:44:34] ../src/tree/updater_gpu_hist.cu:712: Exception in gpu_hist: [10:44:34] ../src/c_api/../data/../common/device_helpers.cuh:428: Memory allocation error on worker 0: std::bad_alloc: cudaErrorMemoryAllocation: out of memory
- Free memory: 125370368
- Requested memory: 201359360

Stack trace:
  [bt] (0) /home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x38f939) [0x7f0dd2f8f939]
  [bt] (1) /home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x393d4b) [0x7f0dd2f93d4b]
  [bt] (2) /home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x402cba) [0x7f0dd3002cba]
  [bt] (3) /home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x403278) [0x7f0dd3003278]
  [bt] (4) /home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x3f536c) [0x7f0dd2ff536c]
  [bt] (5) /home/texs/anac

In [ ]:
import csv

path = os.path.join(RESULTS_PATH, 'har_kfold_{}.csv'.format('_'.join(AUGMENTATIONS)))
with open(path, 'w', newline='') as csvfile:
    row = ['Sensors', 'f1 train', 'f1 test']
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(row)
    for name in names_comb:
        row = [name]
        f1_mean_tr = np.array([ f1[0] for f1 in components_map[name]]).mean()
        f1_stds_tr = np.array([ f1[0] for f1 in components_map[name]]).std()
        f1_mean_te = np.array([ f1[1] for f1 in components_map[name]]).mean()
        f1_stds_te = np.array([ f1[1] for f1 in components_map[name]]).std()
        f1_mean_te_rot = np.array([ f1[2] for f1 in components_map[name]]).mean()
        f1_stds_te_rot = np.array([ f1[2] for f1 in components_map[name]]).std()
        f1_mean_te_per = np.array([ f1[3] for f1 in components_map[name]]).mean()
        f1_stds_te_per = np.array([ f1[3] for f1 in components_map[name]]).std()
        f1_mean_te_tim = np.array([ f1[4] for f1 in components_map[name]]).mean()
        f1_stds_te_tim = np.array([ f1[4] for f1 in components_map[name]]).std()
        f1_mean_te_mag = np.array([ f1[5] for f1 in components_map[name]]).mean()
        f1_stds_te_mag = np.array([ f1[5] for f1 in components_map[name]]).std()
        f1_mean_te_sca = np.array([ f1[6] for f1 in components_map[name]]).mean()
        f1_stds_te_sca = np.array([ f1[6] for f1 in components_map[name]]).std()
        f1_mean_te_jit = np.array([ f1[7] for f1 in components_map[name]]).mean()
        f1_stds_te_jit = np.array([ f1[7] for f1 in components_map[name]]).std()
        
        row = [
            name, 
            '{:.3f} ({:.3f})'.format(f1_mean_tr, f1_stds_tr), 
            '{:.3f} ({:.3f})'.format(f1_mean_te, f1_stds_te), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_rot, f1_stds_te_rot), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_per, f1_stds_te_per), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_tim, f1_stds_te_tim), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_mag, f1_stds_te_mag), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_sca, f1_stds_te_sca), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_jit, f1_stds_te_jit), 
        ]
        print(row)
        spamwriter.writerow(row)


[' Acc', '1.000 (0.000)', '0.828 (0.000)', '0.801 (0.000)', '0.710 (0.000)', '0.486 (0.000)', '0.807 (0.000)', '0.816 (0.000)', '0.742 (0.000)']
[' Gyro', '1.000 (0.000)', '0.748 (0.000)', '0.738 (0.000)', '0.673 (0.000)', '0.484 (0.000)', '0.746 (0.000)', '0.755 (0.000)', '0.603 (0.000)']
[' Acc Gyro', '1.000 (0.000)', '0.844 (0.000)', '0.821 (0.000)', '0.754 (0.000)', '0.518 (0.000)', '0.833 (0.000)', '0.836 (0.000)', '0.794 (0.000)']


In [ ]:
'Centered  0.966 (0.017)'

'Centered  0.966 (0.017)'

In [ ]:
'Rotation 0.999 (0.001)', '0.955 (0.027)'

('Rotation 0.999 (0.001)', '0.955 (0.027)')

In [ ]:
'Original Acc', '0.998 (0.001)', '0.925 (0.045)'

('Original Acc', '0.998 (0.001)', '0.925 (0.045)')